### This is a simple LGB baseline. You can work for feature engineering.
### The seed is 42, which will bring good luck!


In [3]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import time
import datetime
from numba import jit
from lightgbm import LGBMRegressor
from multiprocessing import Pool
from sklearn.model_selection import TimeSeriesSplit

import pickle
import gc

from tqdm import tqdm

n_fold = 10
group_gap = 31
seed = 42


TRAIN_MARKET_PATH = '/content/drive/MyDrive/hku_qis/hku-qids-2023-quantitative-investment-competition/first_round_train_market_data.csv'
TRAIN_FUNADMENTAL_PATH = '/content/drive/MyDrive/hku_qis/hku-qids-2023-quantitative-investment-competition/first_round_train_fundamental_data.csv'
TRAIN_RETURN_PATH = '/content/drive/MyDrive/hku_qis/hku-qids-2023-quantitative-investment-competition/first_round_train_return_data.csv'


TEST_MARKET_PATH = '/content/drive/MyDrive/hku_qis/hku-qids-2023-quantitative-investment-competition/first_round_test_market_data.csv'
TEST_FUNADMENTAL_PATH = '/content/drive/MyDrive/hku_qis/hku-qids-2023-quantitative-investment-competition/first_round_test_fundamental_data.csv'


pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 350)

OSError: dlopen(/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

In [ ]:
df_train_market = pd.read_csv(TRAIN_MARKET_PATH)
df_train_return = pd.read_csv(TRAIN_RETURN_PATH)
df_train_fundamental = pd.read_csv(TRAIN_FUNADMENTAL_PATH)

df_test_market = pd.read_csv(TEST_MARKET_PATH)
df_test_fundamental = pd.read_csv(TEST_FUNADMENTAL_PATH)


In [ ]:
def split_time(x):
    df1 = x['date_time'].str.split('d', expand=True)
    df1.columns=['code','s']
    code = df1['code']
    df1 = df1['s'].str.split('p', expand=True)
    df1.columns=['day','time_step']
    df2 = x['date_time'].str.rsplit('p', expand=True)
    df2.columns=['day_s','s']
    df1['day_s'] = df2['day_s']
    df1['code'] = code
    x = pd.concat([x,df1],axis=1)
    
    return x

df_train_market = split_time(df_train_market)
df = pd.merge(df_train_fundamental,df_train_market, left_on='date_time',right_on='day_s')  
df = pd.merge(df,df_train_return, left_on='day_s',right_on='date_time')  

df_test_market = split_time(df_test_market)
test = pd.merge(df_test_fundamental,df_test_market, left_on='date_time',right_on='day_s')  


In [ ]:
df = df.drop_duplicates(subset='day_s', keep='last').reset_index(drop=True)
test = test.drop_duplicates(subset='day_s', keep='last').reset_index(drop=True)


In [ ]:
def correlation(a, train_data):
    
    b = train_data.get_label()
    
    a = np.ravel(a)
    b = np.ravel(b)

    len_data = len(a)
    mean_a = np.sum(a) / len_data
    mean_b = np.sum(b) / len_data
    var_a = np.sum(np.square(a - mean_a)) / len_data
    var_b = np.sum(np.square(b - mean_b)) / len_data

    cov = np.sum((a * b))/len_data - mean_a*mean_b
    corr = cov / np.sqrt(var_a * var_b)

    return 'corr', corr, True

# For CV score calculation
def corr_score(pred, valid):
    len_data = len(pred)
    mean_pred = np.sum(pred) / len_data
    mean_valid = np.sum(valid) / len_data
    var_pred = np.sum(np.square(pred - mean_pred)) / len_data
    var_valid = np.sum(np.square(valid - mean_valid)) / len_data

    cov = np.sum((pred * valid))/len_data - mean_pred*mean_valid
    corr = cov / np.sqrt(var_pred * var_valid)

    return corr

# For CV score calculation
def wcorr_score(pred, valid, weight):
    len_data = len(pred)
    sum_w = np.sum(weight)
    mean_pred = np.sum(pred * weight) / sum_w
    mean_valid = np.sum(valid * weight) / sum_w
    var_pred = np.sum(weight * np.square(pred - mean_pred)) / sum_w
    var_valid = np.sum(weight * np.square(valid - mean_valid)) / sum_w

    cov = np.sum((pred * valid * weight)) / sum_w - mean_pred*mean_valid
    corr = cov / np.sqrt(var_pred * var_valid)

    return corr

In [ ]:
df

In [ ]:
def train_and_evaluate(train,test):
    # Hyperparammeters (just basic)
    params = {
      'objective': 'rmse',  
      'boosting_type': 'gbdt',
      'n_jobs': -1,
      'verbose': -1
    }
    
    # Split features and target
    x = train[[i for i in df.columns if i not in ['date_time_x', 'date_time_y', 'day', 'time_step', 'day_s', 'code', 'date_time','return']]]
    y = train['return']
    
    x_test = test[[i for i in df.columns if i not in ['date_time_x', 'date_time_y', 'day', 'time_step', 'day_s', 'code', 'date_time','return']]]

    oof_predictions = np.zeros(x.shape[0])
    test_predictions = np.zeros(x_test.shape[0])
    scores = []

    # Create a KFold object
    gkf = TimeSeriesSplit(n_splits=n_fold,gap=group_gap)
    for fold, (trn_ind, val_ind) in enumerate(gkf.split(train['day'].values)):
    
        print(f'Training fold {fold + 1}')
        x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]

        train_dataset = lgb.Dataset(x_train, y_train)
        val_dataset = lgb.Dataset(x_val, y_val)
        model = lgb.train(params = params, 
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          num_boost_round = 200, 
                          early_stopping_rounds = 20, 
                          verbose_eval = False,
                          feval = correlation)
        # Add predictions to the out of folds array
        
        oof_predictions[val_ind] = model.predict(x_val)
        
        rmspe_score = corr_score(y_val,oof_predictions[val_ind])
        print(f'Our out of folds corr_score is {rmspe_score}')
        scores.append(rmspe_score)
        test_predictions += model.predict(x_test) 
        
    rmspe_score = corr_score(y, oof_predictions)
    print(scores)
    print(f'Our out of folds corr score is {rmspe_score}')
    
    # Return test predictions
    return test_predictions

In [ ]:
test_predictions = train_and_evaluate(df,test)


In [ ]:
# Save test predictions
test['return'] = test_predictions

prediction = test[['date_time_x','return']]
prediction.columns=['date_time','return']
prediction.to_csv('submission.csv',index = False)

In [ ]:
import pandas as pd
import random 
random.seed(20230206)

SUBMISSION_PATH = '/kaggle/working/submission.csv'

POINT_PER_DAY = 50

class QIDS:
    def __init__(self) -> None:
        self.__submission_path = SUBMISSION_PATH
        self.__current_idx = 0
        self.__predict_idx = 0
        self.__num_of_stocks = 54
        self.__point_per_day = POINT_PER_DAY
        self.__end = False
        self.__current_fundamental_df = None

        self.__fundamental_df = pd.read_csv(TEST_FUNADMENTAL_PATH)
        self.__market_df = pd.read_csv(TEST_MARKET_PATH)
        
        if len(self.__fundamental_df) / self.__num_of_stocks != len(self.__market_df)/ self.__num_of_stocks / self.__point_per_day:
            raise ValueError('The length of fundamental data and market data is not equal.')
        self.__length = len(self.__fundamental_df) / self.__num_of_stocks

        with open(self.__submission_path, 'w') as f:
            f.write('date_time,return\n') 
        
        print('Environment is initialized.')
    
    def is_end(self):
        return self.__end

    # return the fun
    def get_current_market(self):
        if self.__end:
            raise ValueError('The environment has ended.')

        # check if the current index is equal to the predict index
        if self.__current_idx != self.__predict_idx:
            raise ValueError('The current index is not equal to the predict index.')

        # load data of the current day
        fundamental_df = self.__fundamental_df.iloc[self.__current_idx * self.__num_of_stocks: (self.__current_idx + 1) * self.__num_of_stocks]
        market_df = self.__market_df.iloc[self.__current_idx * self.__num_of_stocks * self.__point_per_day: (self.__current_idx + 1) * self.__num_of_stocks * self.__point_per_day]
        
        # update the current index
        self.__current_idx += 1
        self.__current_fundamental_df = fundamental_df.reset_index()
        
        return fundamental_df, market_df

    def input_prediction(self, predict_ds: pd.Series):
        if self.__end:
            raise ValueError('The environment has ended.')

        # check if the current index is equal to the predict index plus 1
        if self.__current_idx != self.__predict_idx + 1:
            raise ValueError('The current index is not equal to the predict index plus 1.')

        # check the length of the predict_ds
        if len(predict_ds) != self.__num_of_stocks:
            raise ValueError('The length of input decisions is wrong.')
        
        # check the type of the predict_ds
        if type(predict_ds) != pd.Series:
            raise TypeError('The type of input decisions is wrong.')
        
        # write the prediction to the submission file
        with open(self.__submission_path, 'a') as f:
            for idx in range(len(predict_ds)):
                f.write(f"{str(self.__current_fundamental_df['date_time'][idx])},{str(predict_ds.iloc[idx])}\n")

                # must follow the stock order
                # f.write(f"s{idx}d{self.__current_idx},{str(predict_ds.iloc[idx])}\n")
        
        self.__predict_idx += 1
        if self.__predict_idx == self.__length:
            self.__end = True
            print('Data Feeding is finished.')
        

# initialize the environment
def make_env():
    if random.random() == 0.8396457911824297:
        return QIDS()
    else:
        raise ImportError('You cannot make this environment twice.')

In [ ]:
#from qids_package.qids import *

env = make_env()

import random 
random.seed(42)

while not env.is_end():
	fundamental_df, market_df = env.get_current_market()
	
	l = []
	for idx in range(54):
		l.append(random.random())
	predict_ds =pd.Series(1)
	
	env.input_prediction(predict_ds)